In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import PyQt5

In [2]:
pmra_outliers= pd.read_csv('pmra_outliers.csv')

In [3]:
pmra_outliers.head()

,Unnamed: 0,source_id,ra,dec,l,b,parallax,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,dr2_radial_velocity,dr2_radial_velocity_error,r_med_geo,r_lo_geo,r_hi_geo,cube_id,pmra_deviation,pmdec_deviation
0,145572,2343051792663454976,9.988743,-28.035375,12.649668,-87.298757,4.889204,315.47690,364.193786,0.014907,-147.380689,0.013941,-46.355637,2.122902,203.731020,203.10910,204.340560,0,-343.797461,127.025179
1,2234418,2318914523136657792,8.259353,-29.074708,6.234956,-85.500016,11.751412,852.61940,-127.881662,0.015425,-69.914325,0.013069,-2.713808,0.216948,84.893135,84.78940,85.005550,0,148.277987,49.558816
2,2235300,2320174597821925504,5.224313,-29.690209,10.261629,-82.814269,12.224113,379.63440,-116.179322,0.033126,-85.228552,0.031795,-6.365917,0.192952,81.603070,81.38137,81.827560,0,136.575647,64.873043
3,2236525,2321900178242305664,5.197157,-28.899680,16.477153,-83.008911,14.723607,113.35716,-124.597912,0.109571,-249.947421,0.117070,37.947906,1.550238,67.946014,67.39255,68.458755,0,144.994237,229.591912
4,2237181,2316954295766878080,4.951248,-30.906005,2.406730,-82.123134,2.956088,194.06468,200.038259,0.015128,-30.805249,0.015542,-23.742700,4.483841,334.139280,332.69025,335.533540,0,-179.641934,10.449740


In [4]:
cubes= pd.read_csv('cubos.csv')

In [5]:
cubes

,Unnamed: 0,r_med,l_med,b_med,pmra_med,pmdec_med,std_pmra,std_pmdec
0,0,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
1,1,187.378088,28.421053,-85.263158,28.027875,-24.087528,85.808060,123.094400
2,2,187.378088,47.368421,-85.263158,30.149371,-17.954832,58.382921,44.824317
3,3,187.378088,66.315789,-85.263158,22.858752,-21.608687,53.615574,35.308682
4,4,187.378088,85.263158,-85.263158,29.330315,-20.488036,78.836805,42.225514
...,...,...,...,...,...,...,...,...
6854,6854,991.069513,274.736842,85.263158,NaN,NaN,NaN,NaN
6855,6855,991.069513,293.684211,85.263158,2.716697,-8.621976,8.697161,9.351986
6856,6856,991.069513,312.631579,85.263158,-0.849081,-9.210974,0.000000,0.000000
6857,6857,991.069513,331.578947,85.263158,NaN,NaN,NaN,NaN


In [6]:
cubes.rename(columns={f'{cubes.columns[0]}': 'cube_id'}, inplace=True)

In [7]:
cubes

,cube_id,r_med,l_med,b_med,pmra_med,pmdec_med,std_pmra,std_pmdec
0,0,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
1,1,187.378088,28.421053,-85.263158,28.027875,-24.087528,85.808060,123.094400
2,2,187.378088,47.368421,-85.263158,30.149371,-17.954832,58.382921,44.824317
3,3,187.378088,66.315789,-85.263158,22.858752,-21.608687,53.615574,35.308682
4,4,187.378088,85.263158,-85.263158,29.330315,-20.488036,78.836805,42.225514
...,...,...,...,...,...,...,...,...
6854,6854,991.069513,274.736842,85.263158,NaN,NaN,NaN,NaN
6855,6855,991.069513,293.684211,85.263158,2.716697,-8.621976,8.697161,9.351986
6856,6856,991.069513,312.631579,85.263158,-0.849081,-9.210974,0.000000,0.000000
6857,6857,991.069513,331.578947,85.263158,NaN,NaN,NaN,NaN


In [8]:
data=pmra_outliers.merge(cubes, on='cube_id') #merging outliers with their cubes

## Clustering

The idea consists in grouping the outliers based on their position and see if there's a movement correlation among the ones that are near each other on the plane (we're not taking distance into acount since the outliers are in the same cube, they're not so far from each other).

In [9]:
data

,Unnamed: 0,source_id,ra,dec,l,b,parallax,parallax_over_error,pmra,pmra_error,...,cube_id,pmra_deviation,pmdec_deviation,r_med,l_med,b_med,pmra_med,pmdec_med,std_pmra,std_pmdec
0,145572,2343051792663454976,9.988743,-28.035375,12.649668,-87.298757,4.889204,315.476900,364.193786,0.014907,...,0,-343.797461,127.025179,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
1,2234418,2318914523136657792,8.259353,-29.074708,6.234956,-85.500016,11.751412,852.619400,-127.881662,0.015425,...,0,148.277987,49.558816,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
2,2235300,2320174597821925504,5.224313,-29.690209,10.261629,-82.814269,12.224113,379.634400,-116.179322,0.033126,...,0,136.575647,64.873043,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
3,2236525,2321900178242305664,5.197157,-28.899680,16.477153,-83.008911,14.723607,113.357160,-124.597912,0.109571,...,0,144.994237,229.591912,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
4,2237181,2316954295766878080,4.951248,-30.906005,2.406730,-82.123134,2.956088,194.064680,200.038259,0.015128,...,0,-179.641934,10.449740,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27266,327246,1359656942198090752,255.861981,46.274529,72.007854,37.501015,0.996803,100.354034,-29.076734,0.011738,...,6748,26.777932,-6.963259,991.069513,66.315789,37.894737,-2.298802,-5.128035,6.600658,9.988654
27267,1402670,1120087136760813184,143.275147,72.480123,139.720019,37.483382,0.979216,67.712300,-31.032814,0.013591,...,6752,29.377110,11.478355,991.069513,142.105263,37.894737,-1.655703,-2.378986,7.731766,5.681591
27268,1506897,6266416868952173056,233.993394,-13.301407,352.836728,33.190451,0.976034,44.830917,-37.588491,0.026773,...,6763,30.307629,8.739459,991.069513,350.526316,37.894737,-7.280862,-5.414850,9.092020,7.683418
27269,1144747,4430421119884154752,234.810426,6.118888,12.901496,44.969711,0.969873,48.953423,-65.569933,0.020320,...,6764,54.256925,13.878931,991.069513,9.473684,47.368421,-11.313008,-3.746934,14.668983,8.343572


In [41]:
cubes

,cube_id,r_med,l_med,b_med,pmra_med,pmdec_med,std_pmra,std_pmdec
0,0,187.378088,9.473684,-85.263158,20.396325,-20.355509,45.445484,37.075094
1,1,187.378088,28.421053,-85.263158,28.027875,-24.087528,85.808060,123.094400
2,2,187.378088,47.368421,-85.263158,30.149371,-17.954832,58.382921,44.824317
3,3,187.378088,66.315789,-85.263158,22.858752,-21.608687,53.615574,35.308682
4,4,187.378088,85.263158,-85.263158,29.330315,-20.488036,78.836805,42.225514
...,...,...,...,...,...,...,...,...
6854,6854,991.069513,274.736842,85.263158,NaN,NaN,NaN,NaN
6855,6855,991.069513,293.684211,85.263158,2.716697,-8.621976,8.697161,9.351986
6856,6856,991.069513,312.631579,85.263158,-0.849081,-9.210974,0.000000,0.000000
6857,6857,991.069513,331.578947,85.263158,NaN,NaN,NaN,NaN


In [281]:
from sklearn.cluster import DBSCAN
from astropy.coordinates import ICRS
from astropy import units as u

#eps: max distance to classified a particle as neighbor
#min_samples: min number of points to consider particle as core
clusters_save= []
for id in cubes.cube_id:
    cube= pd.concat([data.query(f'cube_id == {id}')])

    if len(cube)!=0:
        clusters= DBSCAN(eps=15, min_samples=1).fit(cube.loc[:,['ra', 'dec', 'r_med_geo']])
        cube['label']= clusters.labels_
        cube.reset_index(inplace=True)
        data_cartesian= ICRS(distance= cube['r_med_geo']*u.pc, dec=cube['dec']*u.degree, ra=cube['ra']*u.degree, \
            pm_ra_cosdec= cube['pmra']*u.mas/u.yr, pm_dec= cube['pmdec']*u.mas/u.yr, radial_velocity= cube['dr2_radial_velocity']*u.km/u.s)
        cube['x']= data_cartesian.cartesian.x.value
        cube['y']= data_cartesian.cartesian.y.value
        cube['z']= data_cartesian.cartesian.z.value
        cube['vx']= data_cartesian.velocity.d_x.value
        cube['vy']= data_cartesian.velocity.d_y.value
        cube['vz']= data_cartesian.velocity.d_z.value

        clusters_save.append(cube)

    continue
    

    plt.figure(figsize=(10,5))
    ax1=plt.subplot(1,2,1)
    ax1.set_title(f'Outliers movement, cube {id}')
    color= sns.color_palette()[:len(cube.label)]
    ax1.quiver(cube.ra.mean(), cube.dec.mean(), cube.pmra_med.unique()[0], cube.pmdec_med.unique()[0], color='k', label='pm_med_cube')
    for i, lab in enumerate(cube.label.unique()):
        cluster= cube.query(f'label=={lab}')
        ax1.quiver(cluster.ra, cluster.dec, cluster.pmra, cluster.pmdec, color=color[i])

    ax2=plt.subplot(1,2,2, sharex=ax1, sharey=ax1)
    ax2.set_title(f'Outlier clusters movement, cube {id}')
    color= sns.color_palette()[:len(cube.label)]
    ax2.quiver(cube.ra.mean(), cube.dec.mean(), cube.pmra_med.unique()[0], cube.pmdec_med.unique()[0], color='k', label='pm_med_cube')
    for i, lab in enumerate(cube.label.unique()):
        cluster= cube.query(f'label=={lab}')
        ax2.quiver(cluster.ra.mean(), cluster.dec.mean(), cluster.pmra.mean(), cluster.pmdec.mean(), color=color[i])

    plt.xlabel('ra')
    plt.ylabel('dec')
    plt.legend()
    plt.tight_layout()
    plt.show()

#Each row represent not a star but a cluster classified by dbscan.

In [282]:
clusters_save[1]

,index,Unnamed: 0,source_id,ra,dec,l,b,parallax,parallax_over_error,pmra,...,pmdec_med,std_pmra,std_pmdec,label,x,y,z,vx,vy,vz
0,5,2236635,2322020952722678912,4.900991,-28.166777,22.716259,-82.875518,27.407176,1494.24770,-289.711999,...,-24.087528,85.80806,123.0944,0,31.991793,2.743221,-17.192832,-60.216152,-55.368471,-229.008578
1,6,2236636,2322020952722679168,4.902773,-28.168729,22.699181,-82.876857,27.409031,1361.84610,-290.590454,...,-24.087528,85.80806,123.0944,0,32.038651,2.748244,-17.219470,-60.023706,-55.581029,-229.668080
2,7,2237065,2322574488107786624,6.111377,-27.026399,32.364684,-83.991395,55.824275,2262.27100,665.754872,...,-24.087528,85.80806,123.0944,0,15.859756,1.698104,-8.136397,0.010801,56.834586,4.908696
3,8,2237702,2323327378694777984,5.520927,-26.709094,34.929147,-83.444109,18.340700,210.93533,415.845442,...,-24.087528,85.80806,123.0944,1,48.395214,4.677769,-24.463364,-11.331162,106.699376,16.007454
4,9,2239299,2333676738050041728,2.271617,-27.121733,30.552301,-80.579446,42.059154,1297.78770,692.223867,...,-24.087528,85.80806,123.0944,0,21.134003,0.838343,-10.833447,19.643336,78.820267,-0.254091


In [263]:
len(clusters_save)

3827

In [275]:
data_cartesian= ICRS(distance= clusters_save[1]['r_med_geo']*u.pc, dec=clusters_save[1]['dec']*u.degree, ra=clusters_save[1]['ra']*u.degree, \
                pm_ra_cosdec= clusters_save[1]['pmra']*u.mas/u.yr, pm_dec= clusters_save[1]['pmdec']*u.mas/u.yr, radial_velocity= clusters_save[1]['dr2_radial_velocity']*u.km/u.s)
clusters_save[1]['x']= data_cartesian.cartesian.x.value
clusters_save[1]['y']= data_cartesian.cartesian.y.value
clusters_save[1]['z']= data_cartesian.cartesian.z.value
clusters_save[1]['vx']= data_cartesian.velocity.d_x.value
clusters_save[1]['vy']= data_cartesian.velocity.d_y.value
clusters_save[1]['vz']= data_cartesian.velocity.d_z.value

In [276]:
clusters_save[1]

,index,Unnamed: 0,source_id,ra,dec,l,b,parallax,parallax_over_error,pmra,...,pmdec_med,std_pmra,std_pmdec,label,x,y,z,vx,vy,vz
0,5,2236635,2322020952722678912,4.900991,-28.166777,22.716259,-82.875518,27.407176,1494.24770,-289.711999,...,-24.087528,85.80806,123.0944,0,31.991793,2.743221,-17.192832,-60.216152,-55.368471,-229.008578
1,6,2236636,2322020952722679168,4.902773,-28.168729,22.699181,-82.876857,27.409031,1361.84610,-290.590454,...,-24.087528,85.80806,123.0944,0,32.038651,2.748244,-17.219470,-60.023706,-55.581029,-229.668080
2,7,2237065,2322574488107786624,6.111377,-27.026399,32.364684,-83.991395,55.824275,2262.27100,665.754872,...,-24.087528,85.80806,123.0944,0,15.859756,1.698104,-8.136397,0.010801,56.834586,4.908696
3,8,2237702,2323327378694777984,5.520927,-26.709094,34.929147,-83.444109,18.340700,210.93533,415.845442,...,-24.087528,85.80806,123.0944,1,48.395214,4.677769,-24.463364,-11.331162,106.699376,16.007454
4,9,2239299,2333676738050041728,2.271617,-27.121733,30.552301,-80.579446,42.059154,1297.78770,692.223867,...,-24.087528,85.80806,123.0944,0,21.134003,0.838343,-10.833447,19.643336,78.820267,-0.254091


In [277]:
%matplotlib qt5

In [278]:
plt.figure()
ax=plt.axes(projection='3d')

color= sns.color_palette()[:len(clusters_save[1].label)]
for i,lab in enumerate(clusters_save[1].label.unique()):
    test= clusters_save[1].query(f'`label`=={lab}')
    ax.quiver(test['x'], test['y'], test['z'], test['vx'], test['vy'], test['vz'], length=5, normalize=True, color=color[i])

## Looking for convergence in outliers

Since outliers are clasified in groups, for each label with more than one member, we're gonna look for where is the geometric center when the outliers reach their maximum approachment to each other. For that, we're gonna assume that stars follow straight trajectories for a few million years (impling that grativational perturbations caused by "black holes" ocurred not so long ago).

Once we have the 'where' of these maximum aproachments, we're gonna look for patterns and see if there is any convergence.

In [283]:
T= int(1e5) #Integration time, yr
ts= np.linspace(-T,T, 100)
max_approach_centers= []

for cluster_ in clusters_save:    
    for lab in cluster_.label.unique():
        cluster= cluster_.query(f'`label`=={lab}')
        if len(cluster)>1:
            print(len(cluster))
            centers=[]
            dists= []
            init= pd.concat([cluster.loc[:,'source_id'],cluster.loc[:, ['x', 'y', 'z']], cluster.loc[:, ['vx', 'vy', 'vz']].apply(lambda x: (x*u.km/u.s).to(u.pc/u.yr) , raw=True)], axis=1)
            for t in ts:
                pos_t= pd.DataFrame(init.loc[:, ['x', 'y', 'z']].values + init.loc[:,['vx', 'vy', 'vz']].values*t, columns=['x', 'y','z'])
                center_t= [pos_t.x.mean(), pos_t.y.mean(), pos_t.z.mean()]
                centers.append(np.array(center_t))
                dist= np.sqrt((init.x -center_t[0])**2 + (init.y -center_t[1])**2 + (init.z -center_t[2])**2).sum()
                dists.append(dist)
            max_approach_centers.append(centers[np.argmin(dists)])

3
4
2
2
3
2
2
2
2
2
2
2
2
2
3
2
5
2
4
2
7
2
3
2
2
4
5
7
3
11
3
2
3
10
3
16
2
3
3
4
7
9
2
4
5
2
2
3
6
5
3
5
15
2
7
3
3
2
2
3
2
2
6
6
11
2
6
15
4
16
13
2
17
3
14
5
17
2
13
12
3
10
2
3
13
16
6
16
3
19
3
5
7
3
11
14
2
20
6
21
2
2
27
26
2
15
22
2
4
17
2
15
2
5
3
6
5
26
20
2
2
32
24
3
2
27
2
11
5
2
3
13
2
4
2
29
3
2
28
2
4
2
36
31
27
2
9
19
11
9
2
16
4
31
3
21
11
13
2
2
2
32
20
3
2
26
4
26
25
2
2
2
22
2
23
3
2
5
2
36
6
6
40
57
5
29
9
2
2
34
35
7
2
4
42
34
20
43
25
5
43
38
3
3
8
2
50
5
49
2
2
3
19
2
41
3
3
2
3
30
3
42
54
2
44
2
55
3
2
3
2
62
2
67
5
54
2
47
5
4
40
39
7
52
3
45
2
59
3
2
45
13
2
2
48
3
3
36
18
2
2
40
4
5
14
3
2
2
2
2
2
2
2
2
2
3
39
2
2
2
2
2
48
7
46
9
66
9
69
66
4
12
55
7
72
4
2
4
71
38
2
56
2
53
40
8
3
35
10
2
52
6
2
5
4
59
4
5
2
2
72
2
2
2
58
3
54
2
36
2
6
2
49
2
3
84
3
3
52
4
5
2
3
77
5
4
68
2
3
78
5
2
10
2
2
83
6
16
6
62
2
2
30
2
2
53
7
70
4
2
2
48
7
3
3
3
68
4
5
76
68
4
3
19
42
8
68
3
2
2
2
55
5
2
2
2
34
3
2
46
5
4
40
5
4
57
2
5
2
46
11
2
2
65
13
2
31
10
2
30
3
3
59
2
3
7
2

In [298]:
max_approach_centers= np.array(max_approach_centers)

In [299]:
plt.figure()
ax= plt.axes(projection='3d')
ax.scatter(max_approach_centers[:,0], max_approach_centers[:,1], max_approach_centers[:,2])

It must be taken into account that dbscan clusters not necessarily converge cause its stars are being classified by how close they are but not how similar is their movement.

In [242]:
plt.figure()
plt.plot(ts,dists)

In [113]:
test

,Unnamed: 0,source_id,ra,dec,l,b,parallax,parallax_over_error,pmra,pmra_error,...,pmdec_med,std_pmra,std_pmdec,label,x,y,z,vx,vy,vz
1,2234418,2318914523136657792,8.259353,-29.074708,6.234956,-85.500016,11.751412,852.61940,-127.881662,0.015425,...,-20.355509,45.445484,37.075094,1,73.425922,10.658490,-41.253787,-8.485032,-53.234917,-23.271596
2,2235300,2320174597821925504,5.224313,-29.690209,10.261629,-82.814269,12.224113,379.63440,-116.179322,0.033126,...,-20.355509,45.445484,37.075094,1,70.595419,6.454892,-40.418836,-17.677278,-46.746252,-25.488079
3,2236525,2321900178242305664,5.197157,-28.899680,16.477153,-83.008911,14.723607,113.35716,-124.597912,0.109571,...,-20.355509,45.445484,37.075094,1,59.239961,5.388295,-32.836780,-2.026413,-40.482485,-88.820681
